In [1]:
import cv2
import numpy as np

ModuleNotFoundError: No module named 'cv2'

In [2]:
face_classfier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_extractor(img):
    faces = face_classfier.detectMultiScale(img,1.3,5)
    if faces is ():
        return None
    for(x,y,w,h) in faces:
        cropped_face = img[y:y+h,x:x+w]
        
    return cropped_face
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret,frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame),(200,200))
        file_name_path = 'F://'+ str(count) + '.jpg'
        cv2.imwrite(file_name_path,face)
        cv2.imshow('Face Cropper', face)
    else:
        print("Face not found")
        pass
    if cv2.waitKey(1) == 13 or count == 200:
        break
cap.release()
cv2.destroyAllWindows()
print("Sample collection complete !")

NameError: name 'cv2' is not defined

In [10]:
from keras.applications import VGG16
rows = 224
cols = 224
model = VGG16(weights = 'imagenet' , include_top=False, input_shape= (rows,cols,3))
model.save('face_recog_vgg.h5')
for layer in model.layers:
    layer.trainable = False
for (i,layer) in enumerate(model.layers):
    print(str(i) + "" + layer.__class__.__name__,layer.trainable)

ModuleNotFoundError: No module named 'keras'

In [ ]:
def addlayer(bottom_model , num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)    
    top_model = Dense(512,activation='relu')(top_model)    
    top_model = Dense(num_classes,activation='softmax')(top_model)  
    return top_model

In [ ]:
model.input

In [ ]:
model.layers

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
num_classes = 2
FC_Head = addlayer(model,num_classes)
modelnew=Model(inputs=model.input,outputs=FC_Head)
print(modelnew.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator 

train_data_dir = 'dataset/train/'
validation_data_dir = 'dataset/test/'
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)
train_batchsize = 16
val_batchsize = 10
train_generator =train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=(rows,cols),
                                                  batch_size=train_batchsize,
                                                  class_mode='categorical',
                                                  )
validation_generator =validation_datagen.flow_from_directory(validation_data_dir,
                                                  target_size=(rows,cols),
                                                  batch_size=val_batchsize,
                                                  class_mode='categorical',
                                                  shuffle=False)

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint , EarlyStopping
checkpoint=ModelCheckpoint("face_recog_vgg.h5",monitor="val_loss",mode="min", save_best_only = True,verbose=1)
earlystop=EarlyStopping(monitor='val_loss',min_delta = 0, patience=3,verbose=1,restore_best_weights=True)
callbacks=[earlystop,checkpoint]

modelnew.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

nb_train_samples=1190
nb_validation_samples=170
epoches=1
batch_size=16

history = modelnew.fit_generator(train_generator,steps_per_epoch=nb_train_samples // batch_size,
                                epochs=epoches,callbacks=callbacks,
                                validation_data = validation_generator,
                                validation_steps=nb_validation_samples // batch_size)
modelnew.save("face_recog_vgg.h5")